In [2]:
import pandas as pd
df1=pd.read_parquet('yellow_tripdata_2024-01.parquet', engine='pyarrow')

In [7]:
df2=pd.read_parquet('yellow_tripdata_2024-02.parquet', engine='pyarrow')
df3=pd.read_parquet('yellow_tripdata_2024-03.parquet', engine='pyarrow')
df4=pd.read_parquet('yellow_tripdata_2024-04.parquet', engine='pyarrow')
df5=pd.read_parquet('yellow_tripdata_2024-05.parquet', engine='pyarrow')
df6=pd.read_parquet('yellow_tripdata_2024-06.parquet', engine='pyarrow')
df7=pd.read_parquet('yellow_tripdata_2024-07.parquet', engine='pyarrow')
df8=pd.read_parquet('yellow_tripdata_2024-08.parquet', engine='pyarrow')
df9=pd.read_parquet('yellow_tripdata_2024-09.parquet', engine='pyarrow')
df10=pd.read_parquet('yellow_tripdata_2024-10.parquet', engine='pyarrow')

In [9]:
df= pd.concat([df1, df2,df3,df4,df5,df6,df7,df8,df9,df10], ignore_index = True)

In [11]:
df.shape

(33854980, 19)

In [13]:
df_copy = df.copy

In [14]:
df = df[df["payment_type"].isin([1, 2])]

In [15]:
df.shape

(29599481, 19)

: 